# Comparing Advanced Methods: Choosing Your Champion

Now that you've learned the major algorithms, how do you choose the right one for your task?

## What You'll Learn

By the end of this notebook, you'll understand:
- The tournament analogy: when each algorithm shines
- Head-to-head comparisons: TRPO vs PPO vs SAC vs TD3
- Benchmark results on different environments
- Decision flowchart: picking the right algorithm
- Practical considerations: compute, sample efficiency, implementation
- Running your own comparisons

**Prerequisites:** Notebooks 1-4 (Trust Regions, PPO, SB3, SAC)

**Time:** ~25 minutes

---
## The Big Picture: The Tournament Analogy

```
    ┌────────────────────────────────────────────────────────────────┐
    │          THE RL ALGORITHM TOURNAMENT                           │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  Imagine a sports tournament where athletes compete in        │
    │  different events. Each has their specialty!                  │
    │                                                                │
    │  PPO (The All-Rounder):                                       │
    │    "I'm reliable in any event. Maybe not the fastest,        │
    │     but I won't let you down!"                               │
    │    → Best for: General purpose, stability matters            │
    │                                                                │
    │  SAC (The Efficient Learner):                                 │
    │    "I learn the most from every practice session!            │
    │     Just give me continuous events."                         │
    │    → Best for: Robotics, continuous control                  │
    │                                                                │
    │  TD3 (The Precision Expert):                                  │
    │    "I'm extremely careful and precise in my training.        │
    │     Continuous control is my specialty."                     │
    │    → Best for: When SAC is too exploratory                   │
    │                                                                │
    │  TRPO (The Careful Strategist):                               │
    │    "I guarantee safe progress, but I'm complex to work with."│
    │    → Best for: Research, theoretical guarantees              │
    │                                                                │
    │  NO SINGLE CHAMPION! Each excels in their domain.            │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, Circle, Wedge
import warnings
warnings.filterwarnings('ignore')

try:
    import gymnasium as gym
except ImportError:
    import gym

# Check if Stable-Baselines3 is available
try:
    from stable_baselines3 import PPO, SAC, TD3, A2C
    from stable_baselines3.common.env_util import make_vec_env
    from stable_baselines3.common.evaluation import evaluate_policy
    import torch
    SB3_AVAILABLE = True
    print("✓ Stable-Baselines3 is installed!")
except ImportError:
    SB3_AVAILABLE = False
    print("✗ Stable-Baselines3 not installed.")
    print("  Install with: pip install stable-baselines3[extra]")

In [ ]:
# Visualize algorithm specializations

fig, ax = plt.subplots(figsize=(14, 10))
ax.set_xlim(0, 14)
ax.set_ylim(0, 12)
ax.axis('off')
ax.set_title('The RL Algorithm Tournament: Each Has Their Specialty', fontsize=16, fontweight='bold')

# Algorithm cards
algorithms = [
    ('PPO', 1, 8.5, '#c8e6c9', '#388e3c', 'The All-Rounder', 
     ['✓ Discrete & Continuous', '✓ Very Stable', '✓ Easy to tune', '✗ Less sample efficient']),
    ('SAC', 7.5, 8.5, '#bbdefb', '#1976d2', 'The Efficient Learner',
     ['✓ Sample efficient', '✓ Good exploration', '✓ Auto-tuning', '✗ Continuous only']),
    ('TD3', 1, 3.5, '#fff3e0', '#f57c00', 'The Precision Expert',
     ['✓ Very stable critic', '✓ Deterministic policy', '✓ Sample efficient', '✗ Less exploration']),
    ('TRPO', 7.5, 3.5, '#e1bee7', '#7b1fa2', 'The Careful Strategist',
     ['✓ Monotonic improvement', '✓ Strong theory', '✗ Complex to implement', '✗ Computationally heavy']),
]

for name, x, y, fcolor, ecolor, title, features in algorithms:
    # Card background
    card = FancyBboxPatch((x, y), 5.5, 3.5, boxstyle="round,pad=0.1",
                           facecolor=fcolor, edgecolor=ecolor, linewidth=3)
    ax.add_patch(card)
    
    # Algorithm name
    ax.text(x + 2.75, y + 3, name, ha='center', fontsize=14, fontweight='bold', color=ecolor)
    ax.text(x + 2.75, y + 2.5, title, ha='center', fontsize=10, style='italic')
    
    # Features
    for i, feat in enumerate(features):
        color = '#388e3c' if feat.startswith('✓') else '#d32f2f'
        ax.text(x + 0.3, y + 1.8 - i*0.45, feat, fontsize=9, color=color)

# Central message
ax.text(7, 7, 'CHOOSE BASED ON YOUR TASK!', ha='center', fontsize=12, 
        fontweight='bold', color='#333')

plt.tight_layout()
plt.show()

print("\nNO ALGORITHM IS BEST FOR EVERYTHING!")
print("  The right choice depends on your specific task.")

---
## Quick Reference: Algorithm Overview

```
    ┌────────────────────────────────────────────────────────────────┐
    │              ALGORITHM OVERVIEW                                │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  ┌──────────────────────────────────────────────────────────┐ │
    │  │                ON-POLICY METHODS                          │ │
    │  │  (Use data once, then discard)                           │ │
    │  ├──────────────────────────────────────────────────────────┤ │
    │  │                                                          │ │
    │  │  TRPO: Trust region constraint (KL divergence)           │ │
    │  │        Complex optimization, strong guarantees           │ │
    │  │                                                          │ │
    │  │  PPO:  Clipped objective (simpler than TRPO)             │ │
    │  │        Industry standard, very popular                   │ │
    │  │                                                          │ │
    │  └──────────────────────────────────────────────────────────┘ │
    │                                                                │
    │  ┌──────────────────────────────────────────────────────────┐ │
    │  │               OFF-POLICY METHODS                          │ │
    │  │  (Store and reuse data via replay buffer)                │ │
    │  ├──────────────────────────────────────────────────────────┤ │
    │  │                                                          │ │
    │  │  SAC:  Maximum entropy, stochastic policy                │ │
    │  │        Best exploration, continuous control              │ │
    │  │                                                          │ │
    │  │  TD3:  Twin critics, deterministic policy                │ │
    │  │        Very stable, less exploration than SAC            │ │
    │  │                                                          │ │
    │  └──────────────────────────────────────────────────────────┘ │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Create detailed comparison table
print("ALGORITHM COMPARISON TABLE")
print("="*80)

comparison = {
    'Feature': ['Type', 'Actions', 'Policy', 'Sample Efficiency', 
                'Stability', 'Implementation', 'Use Cases'],
    'PPO': ['On-policy', 'Discrete/Continuous', 'Stochastic', 'Lower', 
            'Very High', 'Easy', 'General, RLHF'],
    'SAC': ['Off-policy', 'Continuous', 'Stochastic', 'High', 
            'High', 'Medium', 'Robotics'],
    'TD3': ['Off-policy', 'Continuous', 'Deterministic', 'High', 
            'Very High', 'Medium', 'Precision control'],
    'TRPO': ['On-policy', 'Discrete/Continuous', 'Stochastic', 'Lower', 
             'High', 'Hard', 'Research'],
}

# Print table
header = f"{'Feature':<20} {'PPO':<15} {'SAC':<15} {'TD3':<15} {'TRPO':<15}"
print(header)
print("-" * 80)

for i in range(len(comparison['Feature'])):
    row = f"{comparison['Feature'][i]:<20} "
    row += f"{comparison['PPO'][i]:<15} "
    row += f"{comparison['SAC'][i]:<15} "
    row += f"{comparison['TD3'][i]:<15} "
    row += f"{comparison['TRPO'][i]:<15}"
    print(row)

print("="*80)

---
## Dimension 1: On-Policy vs Off-Policy

```
    ┌────────────────────────────────────────────────────────────────┐
    │              ON-POLICY vs OFF-POLICY                           │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  ON-POLICY (PPO, TRPO):                                       │
    │    • Data must come from CURRENT policy                       │
    │    • Use once, then throw away                                │
    │    • Need lots of environment interactions                    │
    │    • But: Very stable gradients                               │
    │                                                                │
    │    When is this OK?                                           │
    │      ✓ Simulations are fast/cheap                             │
    │      ✓ You have lots of compute                               │
    │      ✓ Stability > efficiency                                 │
    │                                                                │
    │  OFF-POLICY (SAC, TD3):                                       │
    │    • Can learn from ANY past experience                       │
    │    • Store in replay buffer, reuse many times                 │
    │    • Much more sample efficient                               │
    │    • But: Potential instability from old data                 │
    │                                                                │
    │    When is this better?                                       │
    │      ✓ Real-world data (expensive to collect)                 │
    │      ✓ Limited environment interactions                       │
    │      ✓ Continuous control tasks                               │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Visualize sample efficiency difference

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Sample efficiency comparison
ax1 = axes[0]

# Simulated learning curves (samples needed to reach target performance)
samples = np.linspace(0, 1e6, 100)

# On-policy (PPO) - needs more samples
ppo_perf = 100 * (1 - np.exp(-samples / 3e5))

# Off-policy (SAC) - learns faster
sac_perf = 100 * (1 - np.exp(-samples / 1e5))

ax1.plot(samples/1e6, ppo_perf, 'g-', linewidth=3, label='PPO (on-policy)')
ax1.plot(samples/1e6, sac_perf, 'b-', linewidth=3, label='SAC (off-policy)')

ax1.axhline(y=90, color='red', linestyle='--', alpha=0.5, label='Target performance')

# Mark samples to reach 90%
ppo_samples_90 = -3e5 * np.log(1 - 0.9) / 1e6
sac_samples_90 = -1e5 * np.log(1 - 0.9) / 1e6

ax1.axvline(x=ppo_samples_90, color='green', linestyle=':', alpha=0.7)
ax1.axvline(x=sac_samples_90, color='blue', linestyle=':', alpha=0.7)

ax1.annotate(f'{ppo_samples_90:.2f}M', xy=(ppo_samples_90, 70), fontsize=10, color='green')
ax1.annotate(f'{sac_samples_90:.2f}M', xy=(sac_samples_90, 70), fontsize=10, color='blue')

ax1.set_xlabel('Environment Samples (Millions)', fontsize=11)
ax1.set_ylabel('Performance (%)', fontsize=11)
ax1.set_title('Sample Efficiency: On-Policy vs Off-Policy', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: Data reuse visualization
ax2 = axes[1]
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 10)
ax2.axis('off')
ax2.set_title('Data Reuse', fontsize=12, fontweight='bold')

# On-policy: use once
ax2.text(2.5, 9, 'ON-POLICY', ha='center', fontsize=11, fontweight='bold', color='#388e3c')
for i in range(4):
    box = FancyBboxPatch((0.5 + i*1.2, 7), 1, 1, boxstyle="round,pad=0.05",
                          facecolor='#c8e6c9', edgecolor='#388e3c', linewidth=2)
    ax2.add_patch(box)
    ax2.text(1 + i*1.2, 7.5, f'D{i+1}', ha='center', fontsize=9)
    # Arrow to trash
    ax2.annotate('', xy=(1 + i*1.2, 6.9), xytext=(1 + i*1.2, 6.3),
                 arrowprops=dict(arrowstyle='->', lw=1, color='#666'))
    ax2.text(1 + i*1.2, 5.8, '🗑️', ha='center', fontsize=12)

# Off-policy: reuse many times
ax2.text(2.5, 4.5, 'OFF-POLICY', ha='center', fontsize=11, fontweight='bold', color='#1976d2')

# Replay buffer
buffer = FancyBboxPatch((0.5, 1.5), 4, 2.5, boxstyle="round,pad=0.1",
                         facecolor='#bbdefb', edgecolor='#1976d2', linewidth=2)
ax2.add_patch(buffer)
ax2.text(2.5, 3.7, 'Replay Buffer', ha='center', fontsize=10, fontweight='bold')

for i in range(4):
    mini = FancyBboxPatch((0.7 + i*0.9, 1.8), 0.7, 0.8, boxstyle="round,pad=0.02",
                           facecolor='#90caf9', edgecolor='#1976d2', linewidth=1)
    ax2.add_patch(mini)
    ax2.text(1.05 + i*0.9, 2.2, f'D{i+1}', ha='center', fontsize=8)

# Arrows showing reuse
ax2.annotate('', xy=(6, 2.5), xytext=(4.6, 2.5),
             arrowprops=dict(arrowstyle='->', lw=2, color='#1976d2'))
ax2.annotate('', xy=(4.6, 3), xytext=(6, 3),
             arrowprops=dict(arrowstyle='->', lw=2, color='#1976d2'))

ax2.text(6.5, 2.75, 'Reuse\nmany\ntimes!', ha='left', fontsize=10, color='#1976d2')

plt.tight_layout()
plt.show()

print("\nKEY INSIGHT:")
print(f"  Off-policy methods need ~{ppo_samples_90/sac_samples_90:.1f}x FEWER samples!")
print("  But on-policy is more stable and works with discrete actions.")

---
## Dimension 2: Stochastic vs Deterministic Policy

```
    ┌────────────────────────────────────────────────────────────────┐
    │              STOCHASTIC vs DETERMINISTIC                       │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  STOCHASTIC (PPO, SAC, TRPO):                                 │
    │    π(a|s) outputs a DISTRIBUTION over actions                 │
    │    Sample action from distribution                            │
    │                                                                │
    │    ✓ Natural exploration (randomness built-in)                │
    │    ✓ Works for discrete actions                               │
    │    ✓ Can express uncertainty                                  │
    │    ✗ May have suboptimal mean action                          │
    │                                                                │
    │  DETERMINISTIC (TD3):                                         │
    │    μ(s) outputs a SINGLE action directly                      │
    │    Add exploration noise during training                      │
    │                                                                │
    │    ✓ No variance from policy sampling                         │
    │    ✓ Easier to optimize (gradient through action)             │
    │    ✗ Need external exploration (noise)                        │
    │    ✗ Only continuous actions                                  │
    │                                                                │
    │  SAC's ADVANTAGE:                                             │
    │    Stochastic BUT with entropy bonus                          │
    │    → Gets benefits of both!                                   │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Visualize stochastic vs deterministic policies

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Common setup
x = np.linspace(-3, 3, 100)

# Left: Stochastic (normal distribution)
ax1 = axes[0]
mu, sigma = 0.5, 0.8
y = np.exp(-0.5 * ((x - mu) / sigma) ** 2) / (sigma * np.sqrt(2 * np.pi))
ax1.fill_between(x, y, alpha=0.3, color='#4caf50')
ax1.plot(x, y, 'g-', linewidth=2)
ax1.axvline(x=mu, color='green', linestyle='--', linewidth=2, label=f'μ={mu}')

# Sample points
np.random.seed(42)
samples = np.random.normal(mu, sigma, 20)
ax1.scatter(samples, np.zeros_like(samples) + 0.02, c='green', s=50, zorder=5, alpha=0.7)

ax1.set_xlabel('Action', fontsize=11)
ax1.set_ylabel('Probability Density', fontsize=11)
ax1.set_title('Stochastic Policy (PPO, SAC)\nπ(a|s) = N(μ, σ²)', fontsize=12, fontweight='bold', color='#388e3c')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Middle: Deterministic (single point)
ax2 = axes[1]
ax2.axvline(x=0.5, color='#f57c00', linewidth=4, label='μ(s) = 0.5')
ax2.scatter([0.5], [0.5], s=200, c='#f57c00', zorder=5, edgecolors='black')

# Add noise for exploration (training only)
noisy_samples = 0.5 + np.random.normal(0, 0.2, 10)
ax2.scatter(noisy_samples, np.zeros_like(noisy_samples) + 0.1, c='orange', s=50, alpha=0.5, label='+ Exploration noise')

ax2.set_xlabel('Action', fontsize=11)
ax2.set_ylabel('', fontsize=11)
ax2.set_title('Deterministic Policy (TD3)\nμ(s) = single value', fontsize=12, fontweight='bold', color='#f57c00')
ax2.set_xlim(-3, 3)
ax2.set_ylim(0, 1)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Right: SAC (stochastic + entropy)
ax3 = axes[2]
# Show how SAC maintains higher entropy
mu, sigma_sac = 0.5, 1.0  # Higher variance due to entropy bonus
y_sac = np.exp(-0.5 * ((x - mu) / sigma_sac) ** 2) / (sigma_sac * np.sqrt(2 * np.pi))
ax3.fill_between(x, y_sac, alpha=0.3, color='#2196f3')
ax3.plot(x, y_sac, 'b-', linewidth=2, label='SAC (high entropy)')

# Compare with PPO (lower entropy as training progresses)
sigma_ppo = 0.5
y_ppo = np.exp(-0.5 * ((x - mu) / sigma_ppo) ** 2) / (sigma_ppo * np.sqrt(2 * np.pi))
ax3.plot(x, y_ppo, 'g--', linewidth=2, label='PPO (no entropy bonus)')

ax3.set_xlabel('Action', fontsize=11)
ax3.set_ylabel('Probability Density', fontsize=11)
ax3.set_title('SAC: Entropy Bonus\nKeeps exploration high!', fontsize=12, fontweight='bold', color='#1976d2')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKEY INSIGHT:")
print("  • Stochastic: Exploration built into policy")
print("  • Deterministic: Need to add noise for exploration")
print("  • SAC: Best of both worlds with entropy bonus!")

---
## Dimension 3: Key Innovations

```
    ┌────────────────────────────────────────────────────────────────┐
    │              KEY INNOVATIONS BY ALGORITHM                      │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  TRPO:                                                        │
    │    • KL divergence constraint for safe updates                │
    │    • Natural gradient + conjugate gradient                    │
    │    • Theoretical monotonic improvement guarantee              │
    │                                                                │
    │  PPO:                                                         │
    │    • Clipped surrogate objective                              │
    │    • ratio ∈ [1-ε, 1+ε] bounds policy change                 │
    │    • Simple to implement, works as well as TRPO              │
    │                                                                │
    │  SAC:                                                         │
    │    • Maximum entropy objective (reward + entropy)             │
    │    • Twin critics to prevent Q overestimation                 │
    │    • Automatic temperature (α) tuning                         │
    │                                                                │
    │  TD3:                                                         │
    │    • Twin critics (from SAC paper, but used first here)      │
    │    • Delayed policy updates                                   │
    │    • Target policy smoothing (noise on target actions)        │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Visualize key innovations

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Top-left: TRPO - KL constraint
ax1 = axes[0, 0]
ax1.set_xlim(-3, 3)
ax1.set_ylim(-3, 3)

# Objective contours
x = np.linspace(-3, 3, 50)
y = np.linspace(-3, 3, 50)
X, Y = np.meshgrid(x, y)
Z = -0.5*(X-1)**2 - 0.3*(Y-1)**2 + 2
ax1.contourf(X, Y, Z, levels=15, cmap='RdYlGn', alpha=0.6)

# Current policy and trust region
ax1.scatter([0], [0], s=150, c='blue', zorder=5, edgecolors='black', label='θ_old')
circle = plt.Circle((0, 0), 1.0, fill=False, linestyle='--', color='blue', linewidth=3)
ax1.add_patch(circle)

ax1.set_xlabel('θ₁', fontsize=11)
ax1.set_ylabel('θ₂', fontsize=11)
ax1.set_title('TRPO: KL Constraint\nKL(π_old || π_new) ≤ δ', fontsize=12, fontweight='bold', color='#7b1fa2')
ax1.text(0, -2, 'Trust Region', fontsize=10, ha='center', color='blue')
ax1.set_aspect('equal')

# Top-right: PPO - Clipped objective
ax2 = axes[0, 1]
epsilon = 0.2
ratio = np.linspace(0.5, 1.5, 100)

# Positive advantage
A = 1
unclipped = ratio * A
clipped = np.clip(ratio, 1-epsilon, 1+epsilon) * A
ppo_obj = np.minimum(unclipped, clipped)

ax2.plot(ratio, ppo_obj, 'g-', linewidth=3, label='PPO objective')
ax2.axvline(x=1-epsilon, color='gray', linestyle=':', alpha=0.5)
ax2.axvline(x=1+epsilon, color='gray', linestyle=':', alpha=0.5)
ax2.axvline(x=1, color='black', linestyle='-', alpha=0.3)

ax2.fill_between(ratio, 0, ppo_obj, where=(ratio > 1 + epsilon), alpha=0.2, color='red')
ax2.text(1.35, 1.15, 'Clipped!', fontsize=10, color='red')

ax2.set_xlabel('Probability Ratio r(θ)', fontsize=11)
ax2.set_ylabel('Objective', fontsize=11)
ax2.set_title('PPO: Clipped Objective\nclip(r, 1-ε, 1+ε)', fontsize=12, fontweight='bold', color='#388e3c')
ax2.grid(True, alpha=0.3)

# Bottom-left: SAC - Twin critics
ax3 = axes[1, 0]
steps = np.arange(100)
np.random.seed(42)

# Q values with overestimation
true_q = 50 + 30 * (1 - np.exp(-steps/30))
q1 = true_q + np.random.randn(100) * 5 + 5  # Overestimates
q2 = true_q + np.random.randn(100) * 5 + 3  # Also overestimates
min_q = np.minimum(q1, q2)  # Taking min reduces overestimation

ax3.plot(steps, true_q, 'k-', linewidth=2, label='True Q')
ax3.plot(steps, q1, 'r-', alpha=0.5, linewidth=1, label='Q₁')
ax3.plot(steps, q2, 'b-', alpha=0.5, linewidth=1, label='Q₂')
ax3.plot(steps, min_q, 'g-', linewidth=2, label='min(Q₁, Q₂)')

ax3.set_xlabel('Training Steps', fontsize=11)
ax3.set_ylabel('Q Value', fontsize=11)
ax3.set_title('SAC/TD3: Twin Critics\nmin(Q₁, Q₂) prevents overestimation', fontsize=12, fontweight='bold', color='#1976d2')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Bottom-right: TD3 - Delayed updates
ax4 = axes[1, 1]
ax4.set_xlim(0, 10)
ax4.set_ylim(0, 10)
ax4.axis('off')

ax4.set_title('TD3: Delayed Policy Updates\nUpdate policy every d critic updates', fontsize=12, fontweight='bold', color='#f57c00')

# Critic updates (every step)
ax4.text(1, 8, 'Critic:', fontsize=11, fontweight='bold')
for i in range(6):
    box = FancyBboxPatch((1.5 + i*1.2, 7), 0.8, 0.8, boxstyle="round,pad=0.05",
                          facecolor='#e1bee7', edgecolor='#7b1fa2', linewidth=2)
    ax4.add_patch(box)
    ax4.text(1.9 + i*1.2, 7.4, '✓', ha='center', fontsize=12, color='#7b1fa2')

# Policy updates (every 2 steps)
ax4.text(1, 5, 'Policy:', fontsize=11, fontweight='bold')
for i in range(6):
    if i % 2 == 1:  # Only every 2nd
        box = FancyBboxPatch((1.5 + i*1.2, 4), 0.8, 0.8, boxstyle="round,pad=0.05",
                              facecolor='#fff3e0', edgecolor='#f57c00', linewidth=2)
        ax4.add_patch(box)
        ax4.text(1.9 + i*1.2, 4.4, '✓', ha='center', fontsize=12, color='#f57c00')
    else:
        ax4.text(1.9 + i*1.2, 4.4, '✗', ha='center', fontsize=12, color='#999')

ax4.text(5, 2, 'Why? Critic needs to stabilize\nbefore guiding the policy!', 
         ha='center', fontsize=10, color='#666')

plt.tight_layout()
plt.show()

---
## Decision Flowchart: Choosing Your Algorithm

```
    ┌────────────────────────────────────────────────────────────────┐
    │              ALGORITHM DECISION FLOWCHART                      │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │                    START                                       │
    │                      │                                         │
    │                      ▼                                         │
    │              ┌─────────────┐                                   │
    │              │  Discrete   │                                   │
    │              │  Actions?   │                                   │
    │              └──────┬──────┘                                   │
    │                     │                                          │
    │           YES ◄─────┴─────► NO                                 │
    │            │                │                                  │
    │            ▼                ▼                                  │
    │     ┌───────────┐    ┌─────────────┐                          │
    │     │    PPO    │    │   Sample    │                          │
    │     │ (Default) │    │ Efficiency  │                          │
    │     └───────────┘    │  Matters?   │                          │
    │                      └──────┬──────┘                           │
    │                             │                                  │
    │                   NO ◄──────┴──────► YES                       │
    │                    │                  │                        │
    │                    ▼                  ▼                        │
    │             ┌───────────┐      ┌─────────────┐                 │
    │             │    PPO    │      │ Exploration │                 │
    │             │ (Stable)  │      │ Important?  │                 │
    │             └───────────┘      └──────┬──────┘                 │
    │                                       │                        │
    │                             YES ◄─────┴─────► NO               │
    │                              │                │                │
    │                              ▼                ▼                │
    │                       ┌───────────┐    ┌───────────┐          │
    │                       │    SAC    │    │    TD3    │          │
    │                       │ (Entropy) │    │ (Stable)  │          │
    │                       └───────────┘    └───────────┘          │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Interactive decision helper

def recommend_algorithm(discrete_actions=False, sample_efficiency_important=False, 
                        exploration_important=True, rlhf=False):
    """
    Recommend an RL algorithm based on your task requirements.
    
    Args:
        discrete_actions: True if action space is discrete (games, text)
        sample_efficiency_important: True if environment samples are expensive
        exploration_important: True if task requires good exploration
        rlhf: True if doing RL from human feedback for LLMs
    
    Returns:
        str: Recommended algorithm with reasoning
    """
    
    if rlhf:
        return "PPO", "Standard choice for RLHF. Stable, works with discrete tokens."
    
    if discrete_actions:
        return "PPO", "Best general choice for discrete actions."
    
    # Continuous actions
    if not sample_efficiency_important:
        return "PPO", "Stable and reliable for continuous control when samples are cheap."
    
    # Sample efficiency matters
    if exploration_important:
        return "SAC", "Sample efficient + entropy bonus for exploration."
    else:
        return "TD3", "Sample efficient, very stable, deterministic policy."


print("ALGORITHM RECOMMENDATION EXAMPLES")
print("="*60)

scenarios = [
    ("Atari Game", dict(discrete_actions=True)),
    ("LLM Fine-tuning (RLHF)", dict(rlhf=True)),
    ("Robot Arm (simulation)", dict(sample_efficiency_important=False)),
    ("Real Robot (expensive)", dict(sample_efficiency_important=True, exploration_important=True)),
    ("Drone Control (precise)", dict(sample_efficiency_important=True, exploration_important=False)),
]

for name, kwargs in scenarios:
    algo, reason = recommend_algorithm(**kwargs)
    print(f"\n{name}:")
    print(f"  → {algo}")
    print(f"  Reason: {reason}")

print("\n" + "="*60)

In [ ]:
# Visualize the decision flowchart

fig, ax = plt.subplots(figsize=(14, 12))
ax.set_xlim(0, 14)
ax.set_ylim(0, 14)
ax.axis('off')
ax.set_title('Algorithm Decision Flowchart', fontsize=16, fontweight='bold')

# Start
start = Circle((7, 13), 0.5, facecolor='#e3f2fd', edgecolor='#1976d2', linewidth=3)
ax.add_patch(start)
ax.text(7, 13, 'START', ha='center', va='center', fontsize=10, fontweight='bold')

# Decision 1: Discrete actions?
d1 = FancyBboxPatch((4.5, 10.5), 5, 1.5, boxstyle="round,pad=0.1",
                     facecolor='#fff3e0', edgecolor='#f57c00', linewidth=2)
ax.add_patch(d1)
ax.text(7, 11.25, 'Discrete Actions?', ha='center', va='center', fontsize=11)
ax.annotate('', xy=(7, 12), xytext=(7, 12.4),
             arrowprops=dict(arrowstyle='->', lw=2, color='#666'))

# PPO for discrete
ppo1 = FancyBboxPatch((1, 8.5), 2.5, 1.5, boxstyle="round,pad=0.1",
                       facecolor='#c8e6c9', edgecolor='#388e3c', linewidth=3)
ax.add_patch(ppo1)
ax.text(2.25, 9.25, 'PPO', ha='center', va='center', fontsize=12, fontweight='bold', color='#388e3c')
ax.annotate('', xy=(2.25, 10), xytext=(4.4, 11),
             arrowprops=dict(arrowstyle='->', lw=2, color='#388e3c'))
ax.text(3, 10.7, 'YES', fontsize=10, color='#388e3c')

# Decision 2: Sample efficiency?
d2 = FancyBboxPatch((8, 8), 5, 1.5, boxstyle="round,pad=0.1",
                     facecolor='#fff3e0', edgecolor='#f57c00', linewidth=2)
ax.add_patch(d2)
ax.text(10.5, 8.75, 'Sample Efficiency\nMatters?', ha='center', va='center', fontsize=10)
ax.annotate('', xy=(10.5, 9.6), xytext=(9.6, 10.4),
             arrowprops=dict(arrowstyle='->', lw=2, color='#666'))
ax.text(10.5, 10.3, 'NO', fontsize=10, color='#666')

# PPO for continuous (samples cheap)
ppo2 = FancyBboxPatch((5.5, 5.5), 2.5, 1.5, boxstyle="round,pad=0.1",
                       facecolor='#c8e6c9', edgecolor='#388e3c', linewidth=3)
ax.add_patch(ppo2)
ax.text(6.75, 6.25, 'PPO', ha='center', va='center', fontsize=12, fontweight='bold', color='#388e3c')
ax.annotate('', xy=(6.75, 7.1), xytext=(7.9, 7.9),
             arrowprops=dict(arrowstyle='->', lw=2, color='#388e3c'))
ax.text(7, 7.7, 'NO', fontsize=10, color='#388e3c')

# Decision 3: Exploration important?
d3 = FancyBboxPatch((9.5, 5), 4, 1.5, boxstyle="round,pad=0.1",
                     facecolor='#fff3e0', edgecolor='#f57c00', linewidth=2)
ax.add_patch(d3)
ax.text(11.5, 5.75, 'Exploration\nImportant?', ha='center', va='center', fontsize=10)
ax.annotate('', xy=(11.5, 6.6), xytext=(11.5, 7.9),
             arrowprops=dict(arrowstyle='->', lw=2, color='#666'))
ax.text(11.7, 7.5, 'YES', fontsize=10, color='#666')

# SAC
sac = FancyBboxPatch((8.5, 2.5), 2.5, 1.5, boxstyle="round,pad=0.1",
                      facecolor='#bbdefb', edgecolor='#1976d2', linewidth=3)
ax.add_patch(sac)
ax.text(9.75, 3.25, 'SAC', ha='center', va='center', fontsize=12, fontweight='bold', color='#1976d2')
ax.annotate('', xy=(9.75, 4.1), xytext=(10.5, 4.9),
             arrowprops=dict(arrowstyle='->', lw=2, color='#1976d2'))
ax.text(9.8, 4.6, 'YES', fontsize=10, color='#1976d2')

# TD3
td3 = FancyBboxPatch((11.5, 2.5), 2.5, 1.5, boxstyle="round,pad=0.1",
                      facecolor='#fff3e0', edgecolor='#f57c00', linewidth=3)
ax.add_patch(td3)
ax.text(12.75, 3.25, 'TD3', ha='center', va='center', fontsize=12, fontweight='bold', color='#f57c00')
ax.annotate('', xy=(12.75, 4.1), xytext=(12.5, 4.9),
             arrowprops=dict(arrowstyle='->', lw=2, color='#f57c00'))
ax.text(12.8, 4.6, 'NO', fontsize=10, color='#f57c00')

# Legend
ax.text(2, 3, 'QUICK GUIDE:', fontsize=11, fontweight='bold')
ax.text(2, 2.3, '• Discrete → PPO', fontsize=10, color='#388e3c')
ax.text(2, 1.7, '• Continuous + cheap samples → PPO', fontsize=10, color='#388e3c')
ax.text(2, 1.1, '• Continuous + need efficiency + exploration → SAC', fontsize=10, color='#1976d2')
ax.text(2, 0.5, '• Continuous + need efficiency + stability → TD3', fontsize=10, color='#f57c00')

plt.tight_layout()
plt.show()

---
## Running Side-by-Side Comparisons with SB3

In [ ]:
if SB3_AVAILABLE:
    print("RUNNING ALGORITHM COMPARISON")
    print("="*60)
    print("\nComparing PPO, SAC, and TD3 on Pendulum-v1 (continuous control)")
    print("Note: This is a quick demo. Real benchmarks need more timesteps!\n")
    
    # Environment setup
    env_name = 'Pendulum-v1'
    n_envs = 4
    total_timesteps = 10_000  # Short for demo; use 100k+ for real comparison
    
    results = {}
    
    # PPO
    print("Training PPO...")
    env = make_vec_env(env_name, n_envs=n_envs)
    ppo_model = PPO('MlpPolicy', env, verbose=0)
    ppo_model.learn(total_timesteps=total_timesteps)
    ppo_reward, ppo_std = evaluate_policy(ppo_model, env, n_eval_episodes=10)
    results['PPO'] = (ppo_reward, ppo_std)
    print(f"  PPO: {ppo_reward:.1f} ± {ppo_std:.1f}")
    env.close()
    
    # SAC
    print("Training SAC...")
    env = gym.make(env_name)
    sac_model = SAC('MlpPolicy', env, verbose=0)
    sac_model.learn(total_timesteps=total_timesteps)
    sac_reward, sac_std = evaluate_policy(sac_model, env, n_eval_episodes=10)
    results['SAC'] = (sac_reward, sac_std)
    print(f"  SAC: {sac_reward:.1f} ± {sac_std:.1f}")
    env.close()
    
    # TD3
    print("Training TD3...")
    env = gym.make(env_name)
    td3_model = TD3('MlpPolicy', env, verbose=0)
    td3_model.learn(total_timesteps=total_timesteps)
    td3_reward, td3_std = evaluate_policy(td3_model, env, n_eval_episodes=10)
    results['TD3'] = (td3_reward, td3_std)
    print(f"  TD3: {td3_reward:.1f} ± {td3_std:.1f}")
    env.close()
    
    print("\n" + "="*60)
else:
    print("Install Stable-Baselines3 to run this comparison.")
    results = {
        'PPO': (-800, 100),  # Simulated results
        'SAC': (-400, 80),
        'TD3': (-450, 90),
    }

In [ ]:
# Visualize comparison results

fig, ax = plt.subplots(figsize=(10, 6))

algorithms = list(results.keys())
rewards = [results[a][0] for a in algorithms]
stds = [results[a][1] for a in algorithms]

colors = ['#4caf50', '#2196f3', '#ff9800']
bars = ax.bar(algorithms, rewards, yerr=stds, capsize=10, 
              color=colors, edgecolor='black', linewidth=2)

ax.axhline(y=-200, color='red', linestyle='--', linewidth=2, label='Good performance')

ax.set_ylabel('Mean Reward', fontsize=12)
ax.set_title(f'Algorithm Comparison on Pendulum-v1\n({total_timesteps if SB3_AVAILABLE else "10k"} timesteps)', 
             fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, reward, std in zip(bars, rewards, stds):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 20,
            f'{reward:.0f}', ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("\nINTERPRETATION:")
print("  • Pendulum is continuous control - SAC/TD3 should do well")
print("  • Off-policy methods (SAC, TD3) are more sample efficient")
print("  • For fair comparison, train longer (100k+ timesteps)")

---
## Practical Considerations

```
    ┌────────────────────────────────────────────────────────────────┐
    │              PRACTICAL CONSIDERATIONS                          │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  COMPUTE REQUIREMENTS:                                        │
    │    PPO:  Moderate (on-policy, needs many env interactions)    │
    │    SAC:  Higher memory (replay buffer)                        │
    │    TD3:  Similar to SAC                                       │
    │    TRPO: Highest (second-order optimization)                  │
    │                                                                │
    │  HYPERPARAMETER SENSITIVITY:                                  │
    │    PPO:  Very robust (defaults work well)                     │
    │    SAC:  Robust (auto temperature helps)                      │
    │    TD3:  Moderate                                             │
    │    TRPO: Complex to tune                                      │
    │                                                                │
    │  IMPLEMENTATION COMPLEXITY:                                   │
    │    PPO:  Simple (clipping is easy)                            │
    │    SAC:  Medium (entropy, twin critics)                       │
    │    TD3:  Medium (delayed updates, noise)                      │
    │    TRPO: Hard (natural gradient, line search)                 │
    │                                                                │
    │  REPRODUCIBILITY:                                             │
    │    All algorithms can be sensitive to random seeds!           │
    │    Always run multiple seeds and report mean ± std.           │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Visualize practical considerations as radar chart

fig, ax = plt.subplots(figsize=(10, 8), subplot_kw=dict(projection='polar'))

# Categories
categories = ['Stability', 'Sample\nEfficiency', 'Implementation\nEase', 
              'Discrete\nSupport', 'Exploration', 'Robustness']
N = len(categories)

# Scores (1-5 scale)
scores = {
    'PPO':  [5, 2, 5, 5, 3, 5],
    'SAC':  [4, 5, 3, 1, 5, 4],
    'TD3':  [5, 5, 3, 1, 2, 4],
    'TRPO': [4, 2, 1, 5, 3, 3],
}

# Angles for each category
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]  # Complete the loop

# Plot each algorithm
colors = ['#4caf50', '#2196f3', '#ff9800', '#9c27b0']
for (algo, score), color in zip(scores.items(), colors):
    values = score + score[:1]  # Complete the loop
    ax.plot(angles, values, 'o-', linewidth=2, label=algo, color=color)
    ax.fill(angles, values, alpha=0.1, color=color)

# Set category labels
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, fontsize=10)
ax.set_ylim(0, 5)

ax.set_title('Algorithm Comparison Radar', fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))

plt.tight_layout()
plt.show()

print("\nRADAR CHART INTERPRETATION:")
print("  • Larger area = better overall")
print("  • PPO: Great all-rounder (large, balanced area)")
print("  • SAC: Strong on efficiency and exploration")
print("  • TD3: Strong on efficiency and stability")
print("  • TRPO: Theoretical strengths, practical challenges")

---
## Summary: Key Takeaways

### Algorithm Quick Reference

| Algorithm | Best For | Key Innovation |
|-----------|----------|----------------|
| **PPO** | General purpose, RLHF | Clipped objective |
| **SAC** | Robotics, exploration | Maximum entropy |
| **TD3** | Precision control | Delayed updates |
| **TRPO** | Research | KL constraint |

### Decision Summary

```
Discrete actions?     → PPO
RLHF for LLMs?        → PPO  
Continuous + cheap?   → PPO
Continuous + explore? → SAC
Continuous + stable?  → TD3
```

### Key Tradeoffs

| Tradeoff | On-Policy (PPO) | Off-Policy (SAC/TD3) |
|----------|-----------------|----------------------|
| Sample efficiency | Lower | Higher |
| Stability | Higher | Moderate |
| Memory | Lower | Higher (buffer) |
| Action types | Any | Continuous |

---
## Test Your Understanding

**1. When should you choose PPO over SAC?**
<details>
<summary>Click to reveal answer</summary>
Choose PPO when:
- You have discrete actions (SAC only works with continuous)
- Doing RLHF for language models
- Environment samples are cheap (simulation is fast)
- Stability is more important than sample efficiency
- You want the simplest implementation
</details>

**2. What's the main advantage of off-policy methods?**
<details>
<summary>Click to reveal answer</summary>
Off-policy methods (SAC, TD3) can reuse past experiences stored in a replay buffer. This makes them much more sample efficient - they can learn from the same data multiple times rather than discarding it after one use. This is crucial when environment interactions are expensive (real robots, complex simulations).
</details>

**3. Why does SAC often explore better than TD3?**
<details>
<summary>Click to reveal answer</summary>
SAC uses maximum entropy reinforcement learning, which adds an entropy bonus to the reward. This explicitly encourages the policy to maintain randomness in its actions, leading to better exploration. TD3 uses a deterministic policy and relies on adding external noise, which is less principled and can be harder to tune.
</details>

**4. What makes TRPO impractical compared to PPO?**
<details>
<summary>Click to reveal answer</summary>
TRPO requires:
- Computing the Fisher Information Matrix (expensive)
- Conjugate gradient optimization
- Line search to satisfy KL constraint

PPO achieves similar results with just a clipped objective - much simpler to implement and faster to run, while maintaining the benefits of bounded policy updates.
</details>

**5. For a new RL project, which algorithm should you try first?**
<details>
<summary>Click to reveal answer</summary>
Start with PPO! Reasons:
- Works with both discrete and continuous actions
- Very stable and robust to hyperparameters
- Easy to implement and debug
- Defaults work well out of the box

Then switch to SAC/TD3 if you need sample efficiency with continuous actions, or stay with PPO if stability is paramount.
</details>

---
## Congratulations!

You've completed the **Advanced Algorithms** section! You now understand:

- ✅ Trust region methods and why they matter (TRPO)
- ✅ PPO: The practical, industry-standard algorithm
- ✅ Using Stable-Baselines3 for production RL
- ✅ SAC: Maximum entropy for continuous control
- ✅ How to choose the right algorithm for your task

**Next Steps:**

Move on to **[RLHF](../rlhf/)** to learn how these algorithms (especially PPO!) are used to align Large Language Models with human preferences!

---

*"The best algorithm is the one that works for YOUR problem!"*